## Demonstrate how similar the logistic regression and basic neural network models are in terms of performance. Build and evaluate both models using the same training/testing dataset.

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

# Import our input dataset
diabetes_df = pd.read_csv('diabetes.csv')
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Now that dataset is loaded into Pandas, prepare the data to train both models. With our logistic regression model, there is no preprocessing or scaling required for the data. However, our basic neural network needs the numerical variables standardized. Therefore, we’ll need to keep track of a scaled and unscaled training dataset such that both models have the correct input data in their preferred formats. 

In [2]:
# Remove diabetes outcome target from features data
y = diabetes_df.Outcome
X = diabetes_df.drop(columns = "Outcome")

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

Standardize the numerical variables using Scikit-learn’s StandardScaler class.

In [3]:
# Preprocess numerical data for neural network

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ready to train and evaluate our models!

#### 1. Logistic Regression Model
Logistic regression models can be built using Scikit-learn’s LogisticRegression class in the linear_model module. For our purposes, we’ll use our basic logistic regression parameters, which include:
* The solver parameter is set to 'lbfgs', which is an algorithm for learning and optimization. The particular solver isn’t very important in this example, but note that a number of optimizers exist.
* The max_iter parameter will be set to 200 iterations, which will give the model sufficient opportunity to converge on effective weights

In [4]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver = "lbfgs", max_iter = 200)

# Train the model
log_classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test, y_pred):.3f}")

 Logistic regression model accuracy: 0.729


#### 2. Basic Neural Network Model. 

Build, compile, and evaluate our basic neural network model using typical binary classifier parameters:
* Our single hidden layer will have an input_dim equal to 8, 16 neuron units, and will use the relu activation function.
* The loss function should be binary_crossentropy, using the adam optimizer.
* Our model should provide the additional accuracy scoring metric and train over a maximum of 50 epochs.

In [5]:
# Define the basic neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units = 16, 
                                   activation = "relu", 
                                   input_dim = 8))
nn_model.add(tf.keras.layers.Dense(units = 1, 
                                   activation = "sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss = "binary_crossentropy", 
                 optimizer = "adam", 
                 metrics = ["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train.to_numpy(), epochs = 100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test.to_numpy(), verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 576 samples
Epoch 1/100
576/576 [==============================] - 1s 1ms/sample - loss: 0.7778 - accuracy: 0.4184
Epoch 2/100
576/576 [==============================] - 0s 65us/sample - loss: 0.7271 - accuracy: 0.4983
Epoch 3/100
576/576 [==============================] - 0s 63us/sample - loss: 0.6852 - accuracy: 0.5833
Epoch 4/100
576/576 [==============================] - 0s 65us/sample - loss: 0.6485 - accuracy: 0.6545
Epoch 5/100
576/576 [==============================] - 0s 65us/sample - loss: 0.6189 - accuracy: 0.7118
Epoch 6/100
576/576 [==============================] - 0s 65us/sample - loss: 0.5940 - accuracy: 0.7448
Epoch 7/100
576/576 [==============================] - 0s 65us/sample - loss: 0.5735 - accuracy: 0.7552
Epoch 8/100
576/576 [==============================] - 0s 67us/sample - loss: 0.5558 - accuracy: 0.7622
Epoch 9/100
576/576 [==============================] - 0s 64us/sample - loss: 0.5410 - accuracy: 0.7691
Epoch 10/100
576/576 [======================

576/576 [==============================] - 0s 69us/sample - loss: 0.4277 - accuracy: 0.8073
Epoch 79/100
576/576 [==============================] - 0s 79us/sample - loss: 0.4273 - accuracy: 0.8073
Epoch 80/100
576/576 [==============================] - 0s 73us/sample - loss: 0.4269 - accuracy: 0.8090
Epoch 81/100
576/576 [==============================] - 0s 83us/sample - loss: 0.4268 - accuracy: 0.8073
Epoch 82/100
576/576 [==============================] - 0s 85us/sample - loss: 0.4264 - accuracy: 0.8073
Epoch 83/100
576/576 [==============================] - 0s 147us/sample - loss: 0.4261 - accuracy: 0.8073
Epoch 84/100
576/576 [==============================] - 0s 131us/sample - loss: 0.4259 - accuracy: 0.8038
Epoch 85/100
576/576 [==============================] - 0s 137us/sample - loss: 0.4258 - accuracy: 0.8056
Epoch 86/100
576/576 [==============================] - 0s 106us/sample - loss: 0.4257 - accuracy: 0.8056
Epoch 87/100
576/576 [==============================] - 0s 87us/

NOTE: There was a ValueError when fitting a neural network. Converted the y_train to a numpy array with .tonumpy().

#### Factors to consider when selecting a model for your data. 
* Neural networks are prone to overfitting and can be more difficult to train than a straightforward logistic regression model. Therefore, if you are trying to build a classifier with limited data points (typically fewer than a thousand data points), or if your dataset has only a few features, neural networks may be overcomplicated.

* Logistic regression models are easier to dissect and interpret than their neural network counterparts, which tends to put more traditional data scientists and non-data experts at ease. 

* In contrast, neural networks (and especially deep neural networks) thrive in large datasets. Datasets with thousands of data points, or datasets with complex features, may overwhelm the logistic regression model, while a deep learning model can evaluate every interaction within and across neurons. 

* Ultimately, the decision between using a logistic regression model and basic neural network model is nuanced and a matter of preference for the data scientist.